In [92]:
import torch
import torch.nn.functional as F
import nlpaug.augmenter.word as naw
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [93]:
text = "Fears for T N pension after talks Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul."
aug = naw.ContextualWordEmbsAug(action="insert")
augmented_text = aug.augment(text, n=4)
augmented_text

["fears for t n williams pension continued after talks unions representing workers include at save turner newall say they finally are'disappointed'after these talks began with depression stricken parent oil firm federal mogul.",
 "shareholder fears rise for the t n pension after talks unions in representing workers at turner draper newall say although they are'disappointed'left after talks with stricken parent of firm of federal mogul.",
 "fears for t · n pension after talks unions representing uk workers but at turner newall could say either they are'disappointed'dropped after public talks ended with the stricken parent firm federal mogul.",
 "final fears for t a n pension emerge after talks unions representing workers at former turner newall say they both are'deeply disappointed'shortly after agreement talks begin with stricken parent firm federal mogul."]

In [94]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("mosesju/distilbert-base-uncased-finetuned-news")
model = AutoModelForSequenceClassification.from_pretrained("mosesju/distilbert-base-uncased-finetuned-news")
model.to(torch.device(device))

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [95]:
input_logits = []
for input_text in [text] + augmented_text:
    input_logits.append(model(**tokenizer(input_text, return_tensors="pt").to(device))[0][0])

input_logits = torch.stack(input_logits)
display(input_logits)
display()

tensor([[ 0.2890, -4.1149,  3.5764, -1.7495],
        [-0.0573, -3.7940,  3.6894, -1.6535],
        [-0.1671, -3.6222,  3.4594, -1.3398],
        [ 0.0566, -3.7586,  3.4811, -1.5894],
        [-0.3797, -3.5373,  3.6425, -1.3264]], device='cuda:0',
       grad_fn=<StackBackward0>)

In [96]:
mean_logits = input_logits.mean(dim=0)
mean_logits

tensor([-0.0517, -3.7654,  3.5698, -1.5317], device='cuda:0',
       grad_fn=<MeanBackward1>)

In [97]:
mean_logits.argmax()

tensor(2, device='cuda:0')

In [98]:
F.softmax(input_logits, dim=-1)

tensor([[3.5823e-02, 4.3809e-04, 9.5907e-01, 4.6653e-03],
        [2.2934e-02, 5.4656e-04, 9.7187e-01, 4.6476e-03],
        [2.5692e-02, 8.1146e-04, 9.6554e-01, 7.9523e-03],
        [3.1324e-02, 6.9017e-04, 9.6195e-01, 6.0402e-03],
        [1.7466e-02, 7.4277e-04, 9.7501e-01, 6.7770e-03]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)